# Déploiement de la solution sur le cloud

Maintenant que nous avons vérifié que notre solution fonctionne, <br />
il est temps de la <u>déployer à plus grande échelle sur un vrai cluster de machines</u>.

**Attention**, *je travaille sous Linux avec une version Ubuntu, <br />
les commandes décrites ci-dessous sont donc réalisées <br />
exclusivement dans cet environnement.*

<u>Plusieurs contraintes se posent</u> :
 1. Quel prestataire de Cloud choisir ?
 2. Quelles solutions de ce prestataire adopter ?
 3. Où stocker nos données ?
 4. Comment configurer nos outils dans ce nouvel environnement ?
 
## 1 Choix du prestataire cloud : AWS

Le prestataire le plus connu et qui offre à ce jour l'offre <br />
la plus large dans le cloud computing est **Amazon Web Services** (AWS).<br />
Certaines de leurs offres sont parfaitement adaptées à notre problématique <br />
et c'est la raison pour laquelle j'utiliserai leurs services.

L'objectif premier est de pouvoir, grâce à AWS, <u>louer de la puissance de calcul à la demande</u>. <br />
L'idée étant de pouvoir, quel que soit la charge de travail, <br />
obtenir suffisamment de puissance de calcul pour pouvoir traiter nos images, <br />
même si le volume de données venait à fortement augmenter.

De plus, la capacité d'utiliser cette puissance de calcul à la demande <br />
permet de diminuer drastiquement les coûts si l'on compare les coûts d'une location <br />
de serveur complet sur une durée fixe (1 mois, 1 année par exemple).

## 2 Choix de la solution technique : EMR

<u>Plusieurs solutions s'offre à nous</u> :
1. Solution **IAAS** (Infrastructure AS A Service)
 - Dans cette configuration **AWS** met à notre disposition des serveurs vierges <br />
   sur lequel nous avons un accès en administrateur, ils sont nommés **instance EC2**.<br />
   Pour faire simple, nous pouvons avec cette solution reproduire pratiquement <br />
   à l'identique la solution mis en œuvre en local sur notre machine.<br />
   <u>On installe nous-même l'intégralité des outils puis on exécute notre script</u> :
  - Installation de **Spark**, **Java** etc.
  - Installation de **Python** (via Anaconda par exemple)
  - Installation de **Jupyter Notebook**
  - Installation des **librairies complémentaires**
  - Il faudra bien évidement veiller à **implémenter les librairies 
    nécessaires à toutes les machines (workers) du cluster**
  - <u>Avantages</u> :
      - Liberté totale de mise en œuvre de la solution
      - Facilité de mise en œuvre à partir d'un modèle qui s'exécute en local sur une machine Linux
  - <u>Inconvénients</u> :
      - Cronophage
          - Nécessité d'installer et de configurer toute la solution
      - Possible problèmes techniques à l'installation des outils (des problématiques qui <br />
        n'existaient pas en local sur notre machine peuvent apparaitre sur le serveur EC2)
      - Solution non pérenne dans le temps, il faudra veiller à la mise à jour des outils <br />
        et éventuellement devoir réinstaller Spark, Java etc. 
2. Solution **PAAS** (Plateforme As A Service)
 - **AWS** fournit énormément de services différents, dans l'un de ceux-là <br />
   il existe une offre qui permet de louer des **instances EC2** <br />
   avec des applications préinstallées et configurées : il s'agit du **service EMR**.
 - **Spark** y sera déjà installé
 - Possibilité de demander l'installation de **Tensorflow** ainsi que **JupyterHub**
 - Possibilité d'indiquer des **packages complémentaires** à installer <br />
   à l'initialisation du serveur **sur l'ensemble des machines du cluster**.
 - <u>Avantages</u> :
     - Facilité de mise en œuvre
         - Il suffit de très peu de configuration pour obtenir <br />
           un environnement parfaitement fonctionnel
     - Rapidité de mise en œuvre
         - Une fois la première configuration réalisée, il est très facile <br />
           et très rapide de recréer des clusters à l'identique qui seront <br />
           disponibles presque instantanément (le temps d'instancier les <br />
           serveurs soit environ 15/20 minutes)
     - Solutions matérielless et logicielles optimisées par les ingénieurs d'AWS
         - On sait que les versions installées vont fonctionner <br />
           et que l'architecture proposée est optimisée
     - Stabilité de la solution
    - Solution évolutive
        Il est facile d’obtenir à chaque nouvelle instanciation une version à jour <br />
        de chaque package, en étant garanti de leur compatibilité avec le reste de l’environnement.
  - Plus sécurisé
	- Les éventuels patchs de sécurité seront automatiquement mis à jour <br />
      à chaque nouvelle instanciation du cluster EMR.
 - <u>Inconvénients</u> :
     - Peut-être un certain manque de liberté sur la version des packages disponibles ? <br />
       Même si je n'ai pas constaté ce problème.
   

Je retiens la solution **PAAS** en choisissant d'utiliser <br />
le service **EMR** d'Amazon Web Services.<br />
Je la trouve plus adaptée à notre problématique et permet <br />
une mise en œuvre qui soit à la fois plus rapide et <br />
plus efficace que la solution IAAS.

## 3 Choix de la solution de stockage des données : Amazon S3

<u>Amazon propose une solution très efficace pour la gestion du stockage des données</u> : **Amazon S3**. <br />
S3 pour Amazon Simple Storage Service.

Il pourrait être tentant de stocker nos données sur l'espace alloué par le serveur **EC2**, <br />
mais si nous ne prenons aucune mesure pour les sauvegarder ensuite sur un autre support, <br />
<u>les données seront perdues</u> lorsque le serveur sera résilié (on résilie le serveur lorsqu'on <br />
ne s'en sert pas pour des raisons de coût).<br />
De fait, si l'on décide d'utiliser l'espace disque du serveur EC2 il faudra imaginer <br />
une solution pour sauvegarder les données avant la résiliation du serveur.
De plus, nous serions exposés à certaines problématiques si nos données venaient à <br />
**saturer** l'espace disponible de nos serveurs (ralentissements, disfonctionnements).

<u>Utiliser **Amazon S3** permet de s'affranchir de toutes ces problématiques</u>. <br />
L'espace disque disponible est **illimité**, et il est **indépendant de nos serveurs EC2**. <br />
L'accès aux données est **très rapide** car nous restons dans l'environnement d'AWS <br />
et nous prenons soin de <u>choisir la même région pour nos serveurs **EC2** et **S3**</u>.

De plus, comme nous le verrons <u>il est possible d'accéder aux données sur **S3** <br />
    de la même manière que l'on **accède aux données sur un disque local**</u>.<br />
Nous utiliserons simplement un **PATH au format s3://...** .

## 4 Configuration de l'environnement de travail

La première étape est d'installer et de configurer [**AWS Cli**](https://aws.amazon.com/fr/cli/),<br />
il s'agit de l'**interface en ligne de commande d'AWS**.<br />
Elle nous permet d'**interagir avec les différents services d'AWS**, comme **S3** par exemple.

Pour pouvoir utiliser **AWS Cli**, il faut le configurer en créant préalablement <br />
un utilisateur à qui on donnera les autorisations dont nous aurons besoin.<br />
Dans ce projet il faut que l'utilisateur ait à minima un contrôle total sur le service S3.

<u>La gestion des utilisateurs et de leurs droits s'effectue via le service **AMI**</u> d'AWS.

Une fois l'utilisateur créé et ses autorisations configurées nous créons une **paire de clés** <br />
qui nous permettra de nous **connecter sans à avoir à devoir saisir systématiquement notre login/mot de passe**.<br />

Il faut également configurer l'**accès SSH** à nos futurs serveurs EC2. <br />
Ici aussi, via un système de clés qui nous dispense de devoir nous authentifier "à la main" à chaque connexion.

Toutes ses étapes de configuration sont parfaitement décrites <br />
dans le cours du projet: [Réalisez des calculs distribués sur des données massives / Découvrez Amazon Web Services](https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308686-decouvrez-amazon-web-services#/id/r-4355822)

Configuration de AWS cli : python -m awscli configure

## 5 Upload de nos données sur S3

Nos outils sont configurés. <br />
Il faut maintenant uploader nos données de travail sur Amazon S3.

Ici aussi les étapes sont décrites avec précision <br />
dans le cours [Réalisez des calculs distribués sur des données massives / Stockez des données sur S3](https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308691-stockez-des-donnees-sur-s3)

Je décide de n'uploader que les données contenues dans le dossier **Test** du [jeu de données du projet](https://www.kaggle.com/moltean/fruits/download)


La première étape consiste à **créer un bucket sur S3** <br />
dans lequel nous uploaderons les données du projet:
- **aws s3 mb s3://data9**

On vérifie que le bucket à bien été créé
- **aws s3 ls**
 - Si le nom du bucket s'affiche alors c'est qu'il a été correctement créé.

On copie ensuite le contenu du dossier "**Test**" <br />
dans un répertoire "**Test**" sur notre bucket "**p8-data**":
1. On se place à l'intérieur du répertoire **Test**
2. **aws sync . s3://data9/Test** (--> python -m awscli s3 sync . s3://p8-leazadi-data/Test)

La commande **sync** est utile pour synchroniser deux répertoires.

<u>Nos données du projet sont maintenant disponibles sur Amazon S3</u>.

## 6 Configuration du serveur EMR

Une fois encore, le cours [Réalisez des calculs distribués sur des données massives / Déployez un cluster de calculs distribués](https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308696-deployez-un-cluster-de-calculs-distribues) <br /> détaille l'essentiel des étapes pour lancer un cluster avec **EMR**.

<u>Je détaillerai ici les étapes particulières qui nous permettent <br />
de configurer le serveur selon nos besoins</u> :

1. Cliquez sur Créer un cluster
![Créer un cluster](img/EMR_creer.png)
2. Cliquez sur Accéder aux options avancées
![Créer un cluster](img/EMR_options_avancees.png)

### 6.1 Étape 1 : Logiciels et étapes

#### 6.1.1 Configuration des logiciels

<u>Sélectionnez les packages dont nous aurons besoin comme dans la capture d'écran</u> :
1. Nous sélectionnons la dernière version d'**EMR**, soit la version **6.3.0** au moment où je rédige ce document
2. Nous cochons bien évidement **Hadoop** et **Spark** qui seront préinstallés dans leur version la plus récente
3. Nous aurons également besoin de **TensorFlow** pour importer notre modèle et réaliser le **transfert learning**
4. Nous travaillerons enfin avec un **notebook Jupyter** via l'application **JupyterHub**<br />
 - Comme nous le verrons dans un instant nous allons <u>paramétrer l'application afin que les notebooks</u>, <br />
   comme le reste de nos données de travail, <u>soient enregistrés directement sur S3</u>.
![Créer un cluster](img/EMR_configuration_logiciels.png)

#### 6.1.2 Modifier les paramètres du logiciel

<u>Paramétrez la persistance des notebooks créés et ouvert via JupyterHub</u> :
- On peut à cette étape effectuer des demandes de paramétrage particulières sur nos applications. <br />
  L'objectif est, comme pour le reste de nos données de travail, <br />
  d'éviter toutes les problématiques évoquées précédemment. <br />
  C'est l'objectif à cette étape, <u>nous allons enregistrer <br />
  et ouvrir les notebooks</u> non pas sur l'espace disque de  l'instance EC2 (comme <br />
  ce serait le cas dans la configuration par défaut de JupyterHub) mais <br />
  <u>directement sur **Amazon S3**</u>.
- <u>deux solutions sont possibles pour réaliser cela</u> :
 1. Créer un **fichier de configuration JSON** que l'on **upload sur S3** et on indique ensuite le chemin d’accès au fichier JSON
 2. Rentrez directement la configuration au format JSON
 
J'ai personnellement créé un fichier JSON lors de la création de ma première instance EMR, <br />
puis lorsqu'on décide de cloner notre serveur pour en recréer un facilement à l'identique, <br />
la configuration du fichier JSON se retrouve directement copié comme dans la capture ci-dessous.

<u>Voici le contenu de mon fichier JSON</u> :  [{"classification":"jupyter-s3-conf","properties":{"s3.persistence.bucket":"p8-data","s3.persistence.enabled":"true"}}]
 Appuyez ensuite sur "**Suivant**"
![Modifier les paramètres du logiciel](img/EMR_parametres_logiciel.png)

### 6.2 Étape 2 : Matériel

A cette étape, laissez les choix par défaut. <br />
<u>L'important ici est la sélection de nos instances</u> :

1. je choisi les instances de type **M5** qui sont des **instances de type équilibrés**
2. je choisi le type **xlarge** qui est l'instance la **moins onéreuse disponible**
 [Plus d'informations sur les instances M5 Amazon EC2](https://aws.amazon.com/fr/ec2/instance-types/m5/)
3. Je sélectionne **1 instance Maître** (le driver) et **2 instances Principales** (les workeurs) <br />
   soit **un total de 3 instance EC2**.
![Choix du materiel](img/EMR_materiel.png)

### 6.3 Étape 3 : Paramètres de cluster généraux

#### 4.6.3.1 Options générales
<u>La première chose à faire est de donner un nom au cluster</u> :<br />
*J'ai également décoché "Protection de la résiliation" pour des raisons pratiques.*
    
![Nom du Cluster](img/EMR_nom_cluster.png)

#### 6.3.2 Actions d'amorçage

Nous allons à cette étape **choisir les packages manquants à installer** et qui <br />
nous serons utiles dans l'exécution de notre notebook.<br />
<u>L'avantage de réaliser cette étape maintenant est que les packages <br />
installés le seront sur l'ensemble des machines du cluster</u>.

La procédure pour créer le fichier **bootstrap** qui contient <br />
l'ensemble des instructions permettant d'installer tous <br />
les packages dont nous aurons besoin est expliqué dans <br />
le cours [Réalisez des calculs distribués sur des données massives / Bootstrapping](https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308696-deployez-un-cluster-de-calculs-distribues#/id/r-4356490)

Nous créons donc un fichier nommé "**bootstrap-emr.sh**" que nous <u>uploadons <br />
sur S3</u>(je l’installe à la racine de mon **bucket "p8-data"**) et nous l'ajoutons <br />
comme indiqué dans la capture d'écran ci-dessous:
![Actions d'amorcage](img/EMR_amorcage.png)

Voici le contenu du fichier **bootstrap-emr.sh**<br />
Comme on peut le constater il s'agit simplement de commande "**pip install**" <br />
pour **installer les bibliothèques manquantes** comme réalisé en local.<br />
Une fois encore, <u>il est nécessaire de réaliser ces actions à cette étape</u> <br />
pour que <u>les packages soient installés sur l'ensemble des machines du cluster</u> <br />
et non pas uniquement sur le driver, comme cela serait le cas si nous exécutions <br />
ces commandes directement dans le notebook JupyterHub ou dans la console EMR (connecté au driver).
![Contenu du fichier bootstrap](img/EMR_bootstrap.png)

**setuptools** et **pip** sont mis à jour pour éviter une problématique <br />
avec l'installation du package **pyarrow**.<br />
**Pandas** a eu droit à une mise à jour majeur (1.3.0) il y a moins d'une semaine <br />
au moment de la rédaction de ce notebook, et la nouvelle version de **Pandas** <br />
nécessite une version plus récente de **Numpy** que la version installée par <br />
défaut (1.16.5) à l'initialisation des instances **EC2**. <u>Il ne semble pas <br />
possible d'imposer une autre version de Numpy que celle installé par <br />
défaut</u> même si on force l'installation d'une version récente de **Numpy** <br />
(en tout cas, ni simplement ni intuitivement).<br />
La mise à jour étant très récente <u>la version de **Numpy** n'est pas encore <br />
mise à jour sur **EC2**</u> mais on peut imaginer que ce sera le cas très rapidement <br />
et il ne sera plus nécessaire d'imposer une version spécifique de **Pandas**.<br />
En attendant, je demande <u>l'installation de l'avant dernière version de **Pandas (1.2.5)**</u>

On clique ensuite sur ***Suivant***

### 6.4 Étape 4 : Sécurité

#### 6.4.1 Options de sécurité

A cette étape nous sélectionnons la **paire de clés EC2** créé précédemment. <br />
Elle nous permettra de se connecter en **ssh** à nos **instances EC2** <br />
sans avoir à entrer nos login/mot de passe.<br />
On laisse les autres paramètres par défaut. <br />
Et enfin, on clique sur "***Créer un cluster***"
 
![EMR Sécurité](img/EMR_securite.png)

## 7 Instanciation du serveur

Il ne nous reste plus qu'à attendre que le serveur soit prêt. <br />
Cette étape peut prendre entre **15 et 20 minutes**.

<u>Plusieurs étapes s'enchaîne, on peut suivre l'avancé du statut du **cluster EMR**</u> :

![Instanciation étape 1](img/EMR_instanciation_01.png)
![Instanciation étape 2](img/EMR_instanciation_02.png)
![Instanciation étape 3](img/EMR_instanciation_03.png)

<u>Lorsque le statut affiche en vert: "**En attente**" cela signifie que l'instanciation <br />
s'est bien déroulée et que notre serveur est prêt à être utilisé</u>. 

## 8 Création du tunnel SSH à l'instance EC2 (Maître)

### 8.1 Création des autorisations sur les connexions entrantes

<u>Nous souhaitons maintenant pouvoir accéder à nos applications</u> :
 - **JupyterHub** pour l'exécution de notre notebook
 - **Serveur d'historique Spark** pour le suivi de l'exécution <br />
   des tâches de notre script lorsqu'il sera lancé
 
Cependant, <u>ces applications ne sont accessibles que depuis le réseau local du driver</u>, <br />
et pour y accéder nous devons **créer un tunnel SSH vers le driver**.

Par défaut, ce driver se situe derrière un firewall qui bloque l'accès en SSH. <br />
<u>Pour ouvrir le port 22 qui correspond au port sur lequel écoute le serveur SSH, <br />
il faut modifier le **groupe de sécurité EC2 du driver**</u>.

Cette étape est décrite dans le cours [Réalisez des calculs distribués sur des données massives / Lancement d'une application à partir du driver](https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308696-deployez-un-cluster-de-calculs-distribues#/id/r-4356512): 

*Il faudra que l'on se connecte en SSH au driver de notre cluster. <br />
Par défaut, ce driver se situe derrière un firewall qui bloque l'accès en SSH. <br />
Pour ouvrir le port 22 qui correspond au port sur lequel écoute le serveur SSH, <br />
il faut modifier le groupe de sécurité EC2 du driver. Sur la page de la console <br />
consacrée à EC2, dans l'onglet "Réseau et sécurité", cliquez sur "Groupes de sécurité". <br />
Vous allez devoir modifier le groupe de sécurité d’ElasticMapReduce-Master. <br />
Dans l'onglet "Entrant", ajoutez une règle SSH dont la source est "N'importe où" <br />
(ou "Mon IP" si vous disposez d'une adresse IP fixe).*

![Configuration autorisation ports entrants pour ssh](img/EMR_config_ssh_01.png)

<u>Une fois cette étape réalisée vous devriez avoir une configuration semblable à la mienne</u> :

![Configuration ssh terminée](img/EMR_config_ssh_02.png)

### 8.2 Création du tunnel ssh vers le Driver

On peut maintenant établir le **tunnel SSH** vers le **Driver**. <br />
Pour cela on récupère les informations de connexion fournis par Amazon <br />
depuis la page du service EMR / Cluster / onglet Récapitulatif en <br />
cliquant sur "**Activer la connexion Web**"

![Activer la connexion Web](img/EMR_tunnel_ssh_01.png)

<u>On récupère ensuite la commande fournis par Amazon pour **établir le tunnel SSH**</u> :

![Récupérer la commande pour établir le tunnel ssh](img/EMR_tunnel_ssh_02.png)

<u>Dans mon cas, la commande ne fonctionne pas tel</u> quel et j'ai du **l'adapter à ma configuration**. <br />
La **clé ssh** se situe dans un dossier "**.ssh**" elle-même située dans <br />
mon **répertoire personnel** dont le symbole est, sous Linux, identifié par un tilde "**~**".

Ayant suivi le cours [Réalisez des calculs distribués sur des données massives / Lancement d'une application à partir du driver](https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives) <br />
j'ai choisi d'utiliser le port **5555** au lieu du **8157**, même si le choix n'est pas très important.<br />
    j'ai également rencontré un <u>problème de compatibilité</u> avec <br />
l'argument "**-N**" (liste des arguments et leur significations <br />
disponibles [ici](https://explainshell.com/explain?cmd=ssh+-L+-N+-f+-l+-D)) j'ai décidé de simplement le supprimer.

<u>Finalement, j'utilise la commande suivante dans un terminal pour établir <br />
    mon tunnel ssh (seul l'URL change d'une instance à une autre)</u> : <br />
"**ssh -i ~/ec2p9.pem -D 8157 hadoop@ec2-13-37-255-214.eu-west-3.compute.amazonaws.com**"

<u>On inscrit "**yes**" pour valider la connexion et si <br />
    la connexion est établit on obtient le résultat suivant</u> :

![Création du tunnel SSH](img/EMR_connexion_ssh_01.png)

Nous avons **correctement établi le tunnel ssh avec le driver** sur le port "8157".

### 8.3 Configuration de FoxyProxy

Une dernière étape est nécessaire pour accéder à nos applications, <br />
en demandant à notre navigateur d'emprunter le tunnel ssh.<br />
J'utilise pour cela **FoxyProxy**.
[Une fois encore, vous pouvez utiliser le cours pour le configurer](https://openclassrooms.com/fr/courses/4297166-realisez-des-calculs-distribues-sur-des-donnees-massives/4308701-realisez-la-maintenance-dun-cluster#/id/r-4356554).

Sinon, ouvrez la configuration de **FoxyProxy** et <u>cliquez sur **Ajouter**</u> en haut à gauche <br />
puis renseigner les éléments comme dans la capture ci-dessous :

![Configuration FoxyProxy Etape 1](img/EMR_foxyproxy_config_01.png)

<u>On obtient le résultat ci-dessous</u> :

![Configuration FoxyProxy Etape 2](img/EMR_foxyproxy_config_02.png)


### 8.4 Accès aux applications du serveur EMR via le tunnel ssh


<u>Avant d'établir notre **tunnel ssh** nous avions ça</u> :

![avant tunnel ssh](img/EMR_tunnel_ssh_avant.png)

<u>On active le **tunnel ssh** comme vu précédemment puis on demande <br />
à notre navigateur de l'utiliser avec **FoxyProxy**</u> :

![FoxyProxy activation](img/EMR_foxyproxy_activation.png)

<u>On peut maintenant s'apercevoir que plusieurs applications nous sont accessibles</u> :

![avant tunnel ssh](img/EMR_tunnel_ssh_apres.png)

## 9 Connexion au notebook JupyterHub

Pour se connecter à **JupyterHub** en vue d'exécuter notre **notebook**, <br />
il faut commencer par <u>cliquer sur l'application **JupyterHub**</u> apparu <br />
depuis que nous avons configuré le **tunnel ssh** et **foxyproxy** sur <br />
notre navigateur (actualisez la page si ce n’est pas le cas).

![Démarrage de JupyterHub](img/EMR_jupyterhub_connexion_01.png)

On passe les éventuels avertissements de sécurité puis <br />
nous arrivons sur une page de connexion.
    
<u>On se connecte avec les informations par défaut</u> :
 - <u>login</u>: **jovyan**
 - <u>password</u>: **jupyter**
 
![Connexion à JupyterHub](img/EMR_jupyterhub_connexion_02.png)

Nous arrivons ensuite dans un dossier vierge de notebook.<br />
Il suffit d'en créer un en cliquant sur "**New**" en haut à droite.

![Liste et création des notebook](img/EMR_jupyterhub_creer_notebooks.png)

Il est également possible d'en <u>uploader un directement dans notre **bucket S3**</u>.

Grace à la <u>**persistance** paramétrée à l'instanciation du cluster <br />
nous sommes actuellement dans l'arborescence de notre **bucket S3**</u>

![Notebook stockés sur S3](img/EMR_jupyterhub_S3.png)

Je décide d'**importer un notebook déjà rédigé en local directement <br />
sur S3** et je l'ouvre depuis **l'interface JupyterHub**.

## 10 Exécution du code

Je décide d'exécuter cette partie du code depuis **JupyterHub hébergé sur notre cluster EMR**.<br />
Pour ne pas alourdir inutilement les explications du **notebook**, je ne réexpliquerai pas les étapes communes <br />
que nous avons déjà vues dans la première partie où l'on a exécuté le code localement sur notre machine virtuelle Ubuntu.

<u>Avant de commencer</u>, il faut s'assurer d'utiliser le **kernel pyspark**.

**En utilisant ce kernel, une session spark est créé à l'exécution de la première cellule**. <br />
Il n'est donc **plus nécessaire d'exécuter le code "spark = (SparkSession ..."** comme lors <br />
de l'exécution de notre notebook en local sur notre VM Ubuntu.

### 10.1 Démarrage de la session Spark

In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1712081113018_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#nettoyer la sortie standard
sc = spark.sparkContext
sc.setLogLevel('ERROR')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<u>Affichage des informations sur la session en cours et liens vers Spark UI</u> :

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1712081113018_0001,pyspark,idle,Link,Link,✔


### 10.2 Installation des packages

Les packages nécessaires ont été installé via l'étape de **bootstrap** à l'instanciation du serveur.

### 10.3 Import des librairies

In [4]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import os
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [9]:
PATH = 's3://data9'
PATH_Data = PATH+'/Test'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p9-habi-data
PATH_Data:   s3://p9-habi-data/Test
PATH_Result: s3://p9-habi-data/Results

### 10.5 Traitement des données

#### 10.5.1 Chargement des données

In [10]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
images.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p9-habi-data...|2024-04-02 17:08:45|125135|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:45|124785|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:46|123514|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:46|122958|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:45|122807|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:45|122654|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:45|122470|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:45|121883|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:46|121883|[FF D8 FF E0 00 1...|
|s3://p9-habi-data...|2024-04-02 17:08:45|121530|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only s

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [12]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(10,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+---------------------------------------------+-----------+
|path                                         |label      |
+---------------------------------------------+-----------+
|s3://p9-habi-data/Test/apple_hit_1/r0_115.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_119.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_107.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_143.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_111.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_127.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_139.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_123.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_151.jpg|apple_hit_1|
|s3://p9-habi-data/Test/apple_hit_1/r0_103.jpg|apple_hit_1|
+---------

#### 10.5.2 Préparation du modèle

In [13]:
# Instanciation of MobileNetV2 model, pre trained with ImageNet database
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [14]:
# New model without the last layer : feature extraction instead of classification
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Weights broadcast
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [17]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation et reduction de dimension PCA à travers l'utilisation de pandas UDF

In [18]:
def nb_composante(dataframe, nb_comp=160):
    '''
    Recherche du nombre de composantes expliquant 95% de la variance
    :param dataframe: Le dataframe d'images
    :param nb_comp: Le nombre de composantes initiales pour l'ACP
    :return: Le nombre de composantes expliquant 95% de la variance totale
    '''

    # Initialisation de l'analyse en composantes principales (ACP)
    pca = PCA(k=nb_comp, inputCol="features_scaled", outputCol="features_pca")

    # Entraînement du modèle PCA
    model_pca = pca.fit(dataframe)

    # Calcul de la variance expliquée
    varexpl = model_pca.explainedVariance * 100



    # Calcul de la variance cumulée
    cumSumVar = varexpl.cumsum()


    # Recherche du nombre de composantes pour atteindre 95% de variance expliquée
    limit = 95
    min_plans = np.argmax(cumSumVar >= limit) + 1
    # Retour du nombre de composantes nécessaires pour expliquer 95% de la variance
    return min_plans

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)
    return 1

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    print("test")
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [20]:
def preprocess_pca(dataframe):
  '''
     Préparation des données :
     - transformation en vecteur dense
     - standardisation
     param : dataframe : dataframe d'images
     return : dataframe avec features vecteur dense standardisé
  '''

  # Préparation des données - conversion des données images en vecteur dense
  transform_vecteur_dense = udf(lambda r: Vectors.dense(r), VectorUDT())
  dataframe = dataframe.withColumn('features_vectors', transform_vecteur_dense('features'))

  # Standardisation obligatoire pour PCA
  scaler_std = StandardScaler(inputCol="features_vectors", outputCol="features_scaled", withStd=True, withMean=True)
  model_std = scaler_std.fit(dataframe)
  # Mise à l'échelle
  dataframe = model_std.transform(dataframe)

  return dataframe

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 10.5.4 Exécutions des actions d'extractions de features

In [21]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://p9-habi-data/Results

In [23]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Réduction de dimension PCA**

In [24]:
from pyspark.ml.feature import PCA
from pyspark.sql.functions import udf
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import StandardScaler

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Pré-processing (vecteur dense, standardisation)
df_pca = preprocess_pca(features_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Nombre de composante expliquant 95% de la variance
n_components = nb_composante(df_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1

In [27]:
# Entrainement de l'algorithme
pca = PCA(k=n_components, inputCol='features_scaled', outputCol='features_pca')
model_pca = pca.fit(df_pca)

# Transformation des images sur les k premières composantes
df_reduit = model_pca.transform(df_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Write the result after PCA to a parquet file
df_reduit.write.mode("overwrite").parquet(PATH_Result + "/pca_results")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
print(PATH_Result + "/pca_results")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://p9-habi-data/Results/pca_results

In [ ]:
df_reduit.write.mode("overwrite").csv(PATH_Result + "/pca_results_csv", header=True)



In [ ]:
print(PATH_Result + "/pca_results_csv")

### 10.6 Chargement des données enregistrées et validation du résultat

In [30]:
df = spark.read.parquet(PATH_Result)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_pca=spark.read.parquet(PATH_Result + "/pca_results")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(path='s3://p9-habi-data/Test/apple_hit_1/r2_211.jpg', label='apple_hit_1', features=[0.0675799772143364, 0.35174116492271423, 0.17365936934947968, 0.0, 0.5641236305236816, 0.20575392246246338, 2.6740732192993164, 0.2215096652507782, 0.48359790444374084, 0.0, 0.0, 0.0, 0.20479105412960052, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04197745770215988, 0.0, 0.5504138469696045, 0.12298305332660675, 0.0, 0.3351147174835205, 0.047496650367975235, 0.0, 0.942962646484375, 0.5146549940109253, 0.0, 0.013633355498313904, 0.0, 0.028960738331079483, 0.06246393173933029, 1.0305652618408203, 0.0, 0.03187152370810509, 0.5550206899642944, 1.7639974355697632, 0.03192978352308273, 0.17166469991207123, 0.0, 0.0, 1.427054524421692, 1.4622894525527954, 0.0, 0.0, 0.4426313638687134, 0.019598787650465965, 1.8614025115966797, 0.30248332023620605, 0.09413401782512665, 0.0022274188231676817, 0.0, 0.0, 0.0, 1.856539011001587, 0.0, 0.42916467785835266, 0.27487289905548096, 0.0, 1.2345383167266846, 0.5567452311515808, 0.0, 0.0

In [33]:
df_pca

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[path: string, label: string, features: array<float>, features_vectors: vector, features_scaled: vector, features_pca: vector]

In [34]:
print(df_pca.printSchema())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- features_vectors: vector (nullable = true)
 |-- features_scaled: vector (nullable = true)
 |-- features_pca: vector (nullable = true)

None

In [35]:
# Sélectionner la colonne "pca_features" et la renommer en "features"
df_pca_features = df_pca.select(col("features_pca").alias("features"))

# Récupérer la première ligne du DataFrame
first_row = df_pca_features.first()

# Obtenir la dimension de la colonne "features"
dimension = len(first_row["features"])
print("Dimension après réduction PCA:", dimension)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dimension après réduction PCA: 1

In [37]:
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['path', 'label', 'features']

In [38]:
df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2671

In [39]:
df_pca.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2671

<u>On peut également constater la présence des fichiers <br />
    au format "**parquet**" sur le **serveur S3**</u> :

![Affichage des résultats sur S3](img/S3_Results.png)

## 11 Suivi de l'avancement des tâches avec le Serveur d'Historique Spark

Il est possible de voir l'avancement des tâches en cours <br />
avec le **serveur d'historique Spark**.

![Accès au serveur d'historique spark](img/EMR_serveur_historique_spark_acces.png)

**Il est également possible de revenir et d'étudier les tâches <br />
qui ont été réalisé, afin de debugger, optimiser les futurs <br />
tâches à réaliser.**

<u>Lorsque la commande "**features_df.write.mode("overwrite").parquet(PATH_Result)**" <br />
était en cours, nous pouvions observer son état d'avancement</u> :

![Progression execution script](img/EMR_jupyterhub_avancement.png)

<u>Le **serveur d'historique Spark** nous permet une vision beaucoup plus précise <br />
de l'exécution des différentes tâche sur les différentes machines du cluster</u> :

![Suivi des tâches spark](img/EMR_SHSpark_01.png)

On peut également constater que notre cluster de calcul a mis <br />
un tout petit peu **moins de 8 minutes** pour traiter les **22 688 images**.

![Temps de traitement](img/EMR_SHSpark_02.png)


## 12 Résiliation de l'instance EMR

Notre travail est maintenant terminé. <br />
Le cluster de machines EMR est **facturé à la demande**, <br />
et nous continuons d'être facturé même lorsque <br />
les machines sont au repos.<br />
Pour **optimiser la facturation**, il nous faut <br />
maintenant **résilier le cluster**.

<u>Je réalise cette commande depuis l'interface AWS</u> :

1. Commencez par **désactiver le tunnel ssh dans FoxyProxy** pour éviter des problèmes de **timeout**.
![Désactivation de FoxyProxy](img/EMR_foxyproxy_desactivation.png)
2. Cliquez sur "**Résilier**"
![Cliquez sur Résilier](img/EMR_resiliation_01.png)
3. Confirmez la résiliation
![Confirmez la résiliation](img/EMR_resiliation_02.png)
4. La résiliation prend environ **1 minute**
![Résiliation en cours](img/EMR_resiliation_03.png)
5. La résiliation est effectuée
![Résiliation terminée](img/EMR_resiliation_04.png)

## 13 Cloner le serveur EMR (si besoin)

Si nous devons de nouveau exécuter notre notebook dans les mêmes conditions, <br />
il nous suffit de **cloner notre cluster** et ainsi en obtenir une copie fonctionnelle <br />
sous 15/20 minutes, le temps de son instanciation.

<u>Pour cela deux solutions</u> :
1. <u>Depuis l'interface AWS</u> :
 1. Cliquez sur "**Cloner**"
   ![Cloner un cluster](img/EMR_cloner_01.png)
 2. Dans notre cas nous ne souhaitons pas inclure d'étapes
   ![Ne pas inclure d'étapes](img/EMR_cloner_02.png)
 3. La configuration du cluster est recréée à l’identique. <br />
    On peut revenir sur les différentes étapes si on souhaite apporter des modifications<br />
    Quand tout est prêt, cliquez sur "**Créer un cluster**"
  ![Vérification/Modification/Créer un cluster](img/EMR_cloner_03.png)
2. <u>En ligne de commande</u> (avec AWS CLI d'installé et de configuré et en s'assurant <br />
   de s'attribuer les droits nécessaires sur le compte AMI utilisé)
 1. Cliquez sur "**Exporter AWS CLI**"
 ![Exporter AWS CLI](img/EMR_cloner_cli_01.png)
 2. Copier/Coller la commande **depuis un terminal**
 ![Copier Coller Commande](img/EMR_cloner_cli_02.png)

## 14 Arborescence du serveur S3 à la fin du projet

<u>Pour information, voici **l'arborescence complète de mon bucket S3 p8-data** à la fin du projet</u> : <br />
*Par soucis de lisibilité, je ne liste pas les 131 sous dossiers du répertoire "Test"*



# Conclusion

Nous avons réalisé ce projet **en deux temps** en tenant <br />
compte des contraintes qui nous ont été imposées.

Nous avons **dans un premier temps développé notre solution en local** <br />
sur une machine virtuelle dans un environnement Linux Ubuntu.

La <u>première phase</u> a consisté à **installer l'environnement de travail Spark**. <br />
**Spark** a un paramètre qui nous permet de travaillé en local et nous permet <br />
ainsi de **simuler du calcul partagé** en considérant <br />
**chaque cœur d'un processeur comme un worker indépendant**.<br />
Nous avons travaillé sur un plus **petit jeu de donnée**, l'idée était <br />
simplement de **valider le bon fonctionnement de la solution**.

Nous avons fait le choix de réaliser du **transfert learning** <br />
à partir du model **MobileNetV2**.<br />
Ce modèle a été retenu pour sa **légèreté** et sa **rapidité d'exécution** <br />
ainsi que pour la **faible dimension de son vecteur en sortie**.

Les résultats ont été enregistrés sur disque en plusieurs <br />
partitions au format "**parquet**".

<u>**La solution a parfaitement fonctionné en mode local**</u>.

La <u>deuxième phase</u> a consisté à créer un **réel cluster de calculs**. <br />
L'objectif était de pouvoir **anticiper une future augmentation de la charge de travail**.

Le meilleur choix retenu a été l'utilisation du prestataire de services **Amazon Web Services** <br />
qui nous permet de **louer à la demande de la puissance de calculs**, <br />
pour un **coût tout à fait acceptable**.<br />
Ce service se nomme **EC2** et se classe parmi les offres **Infrastructure As A Service** (IAAS).

Nous sommes allez plus loin en utilisant un service de plus <br />
haut niveau (**Plateforme As A Service** PAAS)<br />
en utilisant le service **EMR** qui nous permet d'un seul coup <br />
d'**instancier plusieurs serveur (un cluster)** sur lesquels <br />
nous avons pu demander l'installation et la configuration de plusieurs<br />
programmes et librairies nécessaires à notre projet comme **Spark**, <br />
**Hadoop**, **JupyterHub** ainsi que la librairie **TensorFlow**.

En plus d'être plus **rapide et efficace à mettre en place**, nous avons <br />
la **certitude du bon fonctionnement de la solution**, celle-ci ayant été <br />
préalablement validé par les ingénieurs d'Amazon.

Nous avons également pu installer, sans difficulté, **les packages <br />
nécessaires sur l'ensembles des machines du cluster**.

Enfin, avec très peu de modification, et plus simplement encore, <br />
nous avons pu **exécuter notre notebook comme nous l'avions fait localement**.<br />
Nous avons cette fois-ci exécuté le traitement sur **l'ensemble des images de notre dossier "Test"**.

Nous avons opté pour le service **Amazon S3** pour **stocker les données de notre projet**. <br />
S3 offre, pour un faible coût, toutes les conditions dont nous avons besoin pour stocker <br />
et exploiter de manière efficace nos données.<br />
L'espace alloué est potentiellement **illimité**, mais les coûts seront fonction de l'espace utilisé.

Il nous sera **facile de faire face à une monté de la charge de travail** en **redimensionnant** <br />
simplement notre cluster de machines (horizontalement et/ou verticalement au besoin), <br />
les coûts augmenteront en conséquence mais resteront nettement inférieurs aux coûts engendrés <br />
par l'achat de matériels ou par la location de serveurs dédiés.